# Analyzing Raw Bepoz Data

To automate a sync between the bepoz databse and wine_wiki, we need to add some translations between the two data models.
The most useful source of discriminatory data is the comment field, which contains the full wine name, producer, region, 
vintage, volume and more. Unfortunately different formats have emerged over time. While an endgoal would be a formalisation
and correction of the source data, at the moment we will need to define some translation functions. To do this we need to
identify subgroups, seperate them, then define functions which unify the data ready for ingestion into the wine-wiki database.

At this time this notebook is not contained within the wine-wiki project and thus integration with the ORM is not possible,
so we will focus on unification until we can integrate into that project.

## Loading the Data

In [7]:
import duckdb as db

with open(r"queries/cleaning.sql", 'r') as f:
    query = f.read()
    
conn = db.connect()
conn.sql(query)
conn.sql("describe bp_raw;")


┌──────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │ column_type │  null   │   key   │ default │  extra  │
│       varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ product_id           │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ product_name         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ long_name            │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ datetime_last_sale   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ qty_onhand           │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ _comment             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ normal_size1         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ sunday_pricing_size1 │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ public_holiday_size1 │ DOUBLE      │ YES     │ NUL

we will focus on the comment field first, using Product Id as the primary key.


As we can see there are various formats found. We can also see some spelling errors which will need correction.

## Sorting by Whether Vintage is First or Not

An evident discrimnatory factor is whether the string starts with the vintage is first or not.
We can do this with regex.

In [5]:
conn.sql(r"select _comment from raw_comment where regexp_matches(_comment, 'ml\)$');")


┌────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                  _comment                                                  │
│                                                  varchar                                                   │
├────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Albert Bichot Domaine Long-Depaquit Chablis Moutonne Grand Cru Monopole Chardonnay 2022 (750ml)            │
│ A.Christmann IDIG GG Riesling 2021 (750ml)                                                                 │
│ A.Christmann Olberg-Hart GG Riesling 2021 (750ml)                                                          │
│ A.Christmann Vogelsang GG Riesling 2021 (750ml)                                                            │
│ Arnoud Lambert Cremant De Loire (750ml)                                                                    │
│

only 656 entries end with 'ml)'. What do the others end with?


In [6]:
conn.sql(r"select _comment from raw_comment where not regexp_matches(_comment, 'ml\)$');")


┌──────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                               _comment                                               │
│                                               varchar                                                │
├──────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Agrapart Experience Champagne 2012 (750ml).Disgorged 04/2020                                         │
│ 2022 Alkina 'Polygon 3'                                                                              │
│ Alkina 'Poly 5' 20Gr                                                                                 │
│ 2011 Deutz 'Amour de Deutz'. Champagne .Disgorged: 2021                                              │
│ Atallah wines Mclaren Vale Shiraz 2022                                                               │
│ Bass Phillip Estate Pinot Noir 2018 (750ml).         

A mish-mash. We should ignore these for now as they will need to be fixed manually. Although it tooks as though the champagnes will yield to a more sophisticated regex.

In [7]:
conn.sql("from raw_comment limit 10")

┌────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────┐
│ product_id │                                            _comment                                             │
│   int64    │                                             varchar                                             │
├────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────┤
│      31676 │ Albert Bichot Domaine Long-Depaquit Chablis Moutonne Grand Cru Monopole Chardonnay 2022 (750ml) │
│      30438 │ A.Christmann IDIG GG Riesling 2021 (750ml)                                                      │
│      30437 │ A.Christmann Olberg-Hart GG Riesling 2021 (750ml)                                               │
│      30439 │ A.Christmann Vogelsang GG Riesling 2021 (750ml)                                                 │
│      31909 │ Arnoud Lambert Cremant De Loire (750ml)                                          

In [8]:
conn.sql("show tables")

┌───────────────┐
│     name      │
│    varchar    │
├───────────────┤
│ bp_raw        │
│ raw_comment   │
│ vintage_first │
└───────────────┘

## Dividing the Comments into Likes

There are a number of different formats of string, some which will require more work than others. To do this we will copy the 'raw_comment' table then move entries from the copy to their own tables, continuing until 'raw_comment_copy' is empty.

Cleanup includes removing punctuation and whitespace from the ends of the strings. Stock with 0 qty on hand are also excluded, as they are not immediately pertinant to the wine wiki, and this a useful method of removing meta items such as pairing charges and other special menu items.

In [9]:
display(conn.sql("select count(*) from raw_comment_copy;"))
conn.sql("describe raw_comment_copy;")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          725 │
└──────────────┘

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ _comment    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ product_id  │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

### Segregating Champagnes

Champagnes are significantly different, as they do not end with the volume.

In [10]:
conn.sql("select count(*) from champagne;")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           77 │
└──────────────┘

In [11]:
conn.sql("delete from raw_comment_copy where product_id in (select product_id from champagne)")
conn.sql("select count(*) from raw_comment_copy;")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          648 │
└──────────────┘

## Building Volume Field

As we progress in cleaning up the comment string, important information should be moved into dedicated fields, simplifying the remaining comment string. An easy one is the bottle volume. The majority have "(750ml)" in the string, or "(1500ml)", but some do not. First we will add a volume field then fill it out, then seperate entries without a standard volume expression to create specialised logic before merging back in.

In [12]:
conn.sql(r"""
    select
        ext_comment,
        volume
    from (
        with extract_volume as (
                select
                    unnest(regexp_extract(_comment, '^(.*)\((\d{3,4})\s?(ml)?\)',['ext_comment','volume'], 'i')),
                from
                    raw_comment_copy
                )
        select
            ext_comment.trim() as ext_comment,
            volume.trim() as volume
        from 
            extract_volume
        ) 
        """)

┌────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────┐
│                                            ext_comment                                             │ volume  │
│                                              varchar                                               │ varchar │
├────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────┤
│ Albert Bichot Domaine Long-Depaquit Chablis Moutonne Grand Cru Monopole Chardonnay 2022            │ 750     │
│ A.Christmann IDIG GG Riesling 2021                                                                 │ 750     │
│ A.Christmann Olberg-Hart GG Riesling 2021                                                          │ 750     │
│ A.Christmann Vogelsang GG Riesling 2021                                                            │ 750     │
│ Arnoud Lambert Cremant De Loire                                                               

In [18]:
conn.sql(r"""
alter table raw_comment_copy add column if not exists ext_comment varchar;
alter table raw_comment_copy add column if not exists volume varchar;
update raw_comment_copy as a set
    ext_comment = b.ext_comment,
    volume = b.volume
    from 
    (
    select
        product_id,
        ext_comment.nullif('') as ext_comment,
        volume
    from (
        with extract_volume as (
                select
                    product_id,
                    unnest(regexp_extract(_comment, '^(.*)\((\d{3,4}|\d\.\d)\s?(ml|L)?\)',['ext_comment','volume'], 'i')),
                from
                    raw_comment_copy
                )
        select
            product_id,
            ext_comment.trim() as ext_comment,
            volume.trim() as volume
        from 
            extract_volume
        ) 
    ) as b
    where
        a.product_id = b.product_id;

/*
begin brute-force replacements of entries which dont match the above pattern.
Coalesce any null ext_comment.
*/

update
    raw_comment_copy as a
set
    ext_comment = coalesce(_comment, ext_comment);
-- aqua santa
update raw_comment_copy
set volume = 500
where regexp_matches(_comment, 'aqua santa','i');

-- magnum
update
    raw_comment_copy
set 
    volume = 1500
where
    regexp_matches(_comment, 'magnum','i');
update
    raw_comment_copy as a
set
    ext_comment = regexp_replace(b.ext_comment, 'magnum', 'i')
from
    raw_comment_copy as b
where
    a.product_id = b.product_id;

-- all remaining wines are considered to be 750ml
update raw_comment_copy
set volume = 750 
where volume = '';

""")


# finally need to coalesce ext_comment with _comment.

# conn.sql("""

# """)
# display(conn.sql("select * from raw_comment_copy_vol_extract_success where volume = ''"))
# conn.sql("create or replace table raw_comment_copy as select * from raw_comment_copy_vol_extract_success")
display(conn.sql("select * from raw_comment_copy limit 10"))
display(conn.sql("select * from raw_comment_copy where regexp_matches(_comment, 'magnum','i');"))

┌─────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────┬─────────┐
│                                            _comment                                             │ product_id │                                           ext_comment                                           │ volume  │
│                                             varchar                                             │   int64    │                                             varchar                                             │ varchar │
├─────────────────────────────────────────────────────────────────────────────────────────────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────┼─────────┤
│ Albert Bichot Domaine Long-Depaquit Chablis Moutonne Grand Cru Monopole Chardonnay 2022 (750ml) │      31676 │ Alb

┌──────────────────────────────────────────────────────────────────────────┬────────────┬──────────────────────────────────────────────────────────────────────────┬─────────┐
│                                 _comment                                 │ product_id │                               ext_comment                                │ volume  │
│                                 varchar                                  │   int64    │                                 varchar                                  │ varchar │
├──────────────────────────────────────────────────────────────────────────┼────────────┼──────────────────────────────────────────────────────────────────────────┼─────────┤
│ Joh Jos Prum Graacher Himmelreich Kabinett Riesling 2020 Magnum (1500ml) │      25003 │ Joh Jos Prum Graacher Himmelreich Kabinett Riesling 2020 Magnum (1500ml) │ 1500    │
│ Joh Jos Prum Wehlener Sonnenuhr Spatlese Riesling 2020 Magnum (1500ml)   │      25010 │ Joh Jos Prum Wehlener Sonnenuhr Spa

In [14]:
conn.sql("select * from raw_comment_copy where _comment = ''")

┌──────────┬────────────┬─────────────┬─────────┐
│ _comment │ product_id │ ext_comment │ volume  │
│ varchar  │   int64    │   varchar   │ varchar │
├──────────┴────────────┴─────────────┴─────────┤
│                    0 rows                     │
└───────────────────────────────────────────────┘

## Extracting Vintage

regexp_extract has proven to be very powerful, simplifying the process. Next is to extract the vintages.

In [15]:
conn.sql("select * from raw_comment_copy where ext_comment = ''")

┌──────────┬────────────┬─────────────┬─────────┐
│ _comment │ product_id │ ext_comment │ volume  │
│ varchar  │   int64    │   varchar   │ varchar │
├──────────┴────────────┴─────────────┴─────────┤
│                    0 rows                     │
└───────────────────────────────────────────────┘

In [16]:
conn.sql(r"""
    with unnested as (
        select
            product_id,
            ext_comment as input_comment,
            unnest(regexp_extract(ext_comment, '(.*)(\d{4})(.*)', ['before','vintage', 'after']))
        from
            raw_comment_copy
        ),
    combined as (
    select
        product_id,
        input_comment,
        trim(before || ' '|| after) as ext_comment,
        trim(vintage) as vintage
    from 
        unnested
        ),
    coalesced as (
        select
        product_id,
        input_comment,
        coalesce(ext_comment.nullif(''), input_comment) as ext_comment,
        vintage,
        from
        combined
    )
    -- currently 33 entries without an input comment
    select
        *
    from
        coalesced
    where
        ext_comment = ''
""")

┌────────────┬───────────────┬─────────────┬─────────┐
│ product_id │ input_comment │ ext_comment │ vintage │
│   int64    │    varchar    │   varchar   │ varchar │
├────────────┴───────────────┴─────────────┴─────────┤
│                       0 rows                       │
└────────────────────────────────────────────────────┘